In [1]:
##################################################################
#           《Python人工智能编程实践（2024年度版）》开源代码
#-----------------------------------------------------------------
#            @章节号：6.2.1（全连接神经网络的PyTorch实践）                                        
#            @作者：范淼、徐晟桐 
#            @购书链接：暂无
#            @电子邮箱：fm12@tsinghua.org.cn             
#            @官方交流QQ群号：561500762                        
##################################################################

In [2]:
from torch import nn, optim


#设定超参数。
INPUT_SIZE = 784 
HIDDEN_SIZE = 256 
NUM_CLASSES = 10
EPOCHS = 5
BATCH_SIZE = 64
LEARNING_RATE = 1e-3 


class DNN(nn.Module):
    '''
    自定义全连接神经网络类，继承自nn.Module。
    '''
    def __init__(self, input_size, hidden_size, num_classes):
        
        super(DNN, self).__init__()
        
        self.l1 = nn.Linear(input_size, hidden_size) 
        
        self.relu = nn.ReLU()
        
        self.l2 = nn.Linear(hidden_size, num_classes)   
    
    
    def forward(self, input_tensor):
        
        #添加有256个神经元的隐藏层。
        out = self.l1(input_tensor)
        
        #设定激活函数为ReLU。
        out = self.relu(out)
        
        #添加有10个神经元的输出层。
        out = self.l2(out)
        
        return out 
    

#初始化全连接神经网络。
model = DNN(INPUT_SIZE, HIDDEN_SIZE, NUM_CLASSES)

#设定神经网络的损失函数。
criterion = nn.CrossEntropyLoss()

#设定神经网络的优化方法。
optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE) 

In [3]:
import pandas as pd


#使用pandas，读取fashion_mnist的训练和测试数据文件。
train_data = pd.read_csv('../datasets/fashion_mnist/fashion_mnist_train.csv')
test_data = pd.read_csv('../datasets/fashion_mnist/fashion_mnist_test.csv')

#从训练数据中，拆解出训练特征和类别标签。
X_train = train_data[train_data.columns[1:]]
y_train = train_data['label']

#从测试数据中，拆解出测试特征和类别标签。
X_test = test_data[train_data.columns[1:]]
y_test = test_data['label']

In [4]:
from sklearn.preprocessing import StandardScaler


#初始化数据标准化处理器。
ss = StandardScaler()

#标准化训练数据特征。
X_train = ss.fit_transform(X_train)

#标准化测试数据特征。
X_test = ss.transform(X_test)

In [5]:
import torch
from torch.utils.data import TensorDataset, DataLoader


#构建适用于PyTorch模型训练的数据结构。
train_tensor = TensorDataset(torch.tensor(X_train.astype('float32')), torch.tensor(y_train.values))

#构建适用于PyTorch模型训练的数据读取器。
train_loader = DataLoader(dataset = train_tensor, batch_size = BATCH_SIZE, shuffle = True)

n_total_steps = len(train_loader)

#开启模型训练。
model.train()

for epoch in range(EPOCHS):
    for i, (features, labels) in enumerate(train_loader):  
        outputs = model(features)

        loss = criterion(outputs, labels)
        
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() 
        
        if (i+1) % 300 == 0:
             print (f'Epoch [{epoch+1}/{EPOCHS}], Step[{i+1}/{n_total_steps}], Loss: {loss.item():.4f}') 

Epoch [1/5], Step[300/938], Loss: 0.4639
Epoch [1/5], Step[600/938], Loss: 0.1699
Epoch [1/5], Step[900/938], Loss: 0.2741
Epoch [2/5], Step[300/938], Loss: 0.2542
Epoch [2/5], Step[600/938], Loss: 0.3135
Epoch [2/5], Step[900/938], Loss: 0.3883
Epoch [3/5], Step[300/938], Loss: 0.2197
Epoch [3/5], Step[600/938], Loss: 0.2532
Epoch [3/5], Step[900/938], Loss: 0.1187
Epoch [4/5], Step[300/938], Loss: 0.1796
Epoch [4/5], Step[600/938], Loss: 0.1762
Epoch [4/5], Step[900/938], Loss: 0.5039
Epoch [5/5], Step[300/938], Loss: 0.1913
Epoch [5/5], Step[600/938], Loss: 0.1671
Epoch [5/5], Step[900/938], Loss: 0.2392


In [6]:
#构建适用于PyTorch模型测试的数据结构。
test_tensor = TensorDataset(torch.tensor(X_test.astype('float32')), torch.tensor(y_test.values))

#构建适用于PyTorch模型测试的数据读取器。
test_loader = DataLoader(dataset = test_tensor, batch_size = BATCH_SIZE, shuffle = False)

#开启模型测试。
model.eval()

n_correct = 0
n_samples = 0

for features, labels in test_loader:
    outputs = model(features)
    _, predictions = torch.max(outputs.data, 1)
    
    n_samples += labels.size(0)
    n_correct += (predictions == labels).sum().item() 

acc = 100.0 * n_correct / n_samples

print('全连接神经网络（PyTorch版本）在fashion_mnist测试集上的准确率为: %.2f%%。' %acc) 

全连接神经网络（PyTorch版本）在fashion_mnist测试集上的准确率为: 88.69%。
